<div style="position: relative;">
<img src="https://user-images.githubusercontent.com/7065401/98728503-5ab82f80-2378-11eb-9c79-adeb308fc647.png"></img>

<h1 style="color: white; position: absolute; top:27%; left:10%;">
     Secure RESTful APIs using Python
</h1>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:56%; left:10%;">
    David Mertz, Ph.D.
</h3>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:63%; left:10%;">
    Data Scientist
</h3>
</div>

# Authentication

In the last lesson we looked at the need to make HTTP connections **private** in the sense that their content cannot be read by anyone other than the connecting parties.  It is also often desirable to control **permissions** to make certain requests.  

In a very simple case, there may be a limited number of entities (e.g. other microservices) that should be able to utilize a service.  In more stratified situations, services may respond differently based on the credentials of a requester.  Let's look first at the simpler case.  

There are a couple models we might follow for authentication.  One option is to require credentials upon every request.  Another, which will lead into packaged protocols like JSON Web Tokens (JWT) is to use a token that encodes a previous authentication.  Cookies could be used to maintain such a cookie, or it could be included within requests along with other details.

## Setting cookies

We can arrange a service so that it will require an initial call to a path like `/login` with credentials, but later calls to other methods will succeed once that method succeeds.  Let's look at (part of) the server that will generate random secure tokens and both set client cookes and store those cookies on the server for later comparison.  

```python
#!/usr/bin/env python
import secrets
from datetime import datetime
from flask import Flask, request, make_response, jsonify
app = Flask(__name__)

# User/password pairs
authorized = {'Alice': 'alice_pw', 'Bob': 'bob_pw', 'Carlos': 'carlos_pw'}
auth_tokens = dict()

@app.route('/login', methods =['POST'])
def login():
    username = request.form.get('username')
    password = request.form.get('password')

    # Fail in slightly different ways under different bad logins
    if not username or not password:
        hdr = {'WWW-Authenticate' : 'Basic realm="Login required"'}
        return make_response('Verification failed', 401, hdr)
    
    elif username not in authorized:
        hdr = {'WWW-Authenticate' : 'Basic realm="No such user"'}
        return make_response('Verification failed', 401, hdr)
    
    elif password != authorized[username]:
        hdr = {'WWW-Authenticate' : 'Basic realm="Password incorrect"'}
        return make_response('Verification failed', 403, hdr)
    
    else:
        token = secrets.token_urlsafe(16)
        auth_tokens[username] = token
        resp = make_response("Logged in", 200)
        resp.set_cookie('app1-username', username)
        resp.set_cookie('app1-token', token)
        resp.set_cookie('app1-login-timestamp', datetime.now().isoformat())
        return resp
    
if __name__ == "__main__":
    app.run(ssl_context=('pubkey.pem', 'private.pem'), port=5001)
```

Obviously, the management of users and credentials is greatly simplified for this lesson.  In a production micro-service they would not be hard-coded into the server itself, but stored in some kind of database.  Moreover, the server "cookie jar" should likewise use some persistence mechanism that does not rely on the service running continuously.  But in concept, these are the general data you need to maintain on a service in this design.

In [1]:
import requests
requests.packages.urllib3.disable_warnings()
cookies = dict()

In [2]:
resp = requests.post("https://localhost:5001/login", verify='pubkey.pem',
                     data=dict(username="Alice", password="wrong_pw"))

for k, v in resp.headers.items():
    print(f"{k}: {v}")
print()
print(resp.status_code, resp.text)
print(resp.cookies)

WWW-Authenticate: Basic realm="Password incorrect"
Content-Type: text/html; charset=utf-8
Content-Length: 19
Server: Werkzeug/1.0.1 Python/3.8.2
Date: Mon, 19 Apr 2021 02:52:58 GMT

403 Verification failed
<RequestsCookieJar[]>


In [3]:
resp = requests.post("https://localhost:5001/login", verify='pubkey.pem',
                     data=dict(username="Alice", password="alice_pw"))

for k, v in resp.headers.items():
    print(f"{k}: {v}")
print()
print(resp.status_code, resp.text)
print(resp.cookies)

Content-Type: text/html; charset=utf-8
Content-Length: 9
Set-Cookie: app1-username=Alice; Path=/, app1-token=GGTo57n11xqe9cGqo7mzlA; Path=/, app1-login-timestamp=2021-04-18T22:54:04.387689; Path=/
Server: Werkzeug/1.0.1 Python/3.8.2
Date: Mon, 19 Apr 2021 02:54:04 GMT

200 Logged in
<RequestsCookieJar[<Cookie app1-login-timestamp=2021-04-18T22:54:04.387689 for localhost.local/>, <Cookie app1-token=GGTo57n11xqe9cGqo7mzlA for localhost.local/>, <Cookie app1-username=Alice for localhost.local/>]>


In [4]:
cookies.update(resp.cookies)
cookies

{'app1-login-timestamp': '2021-04-18T22:54:04.387689',
 'app1-token': 'GGTo57n11xqe9cGqo7mzlA',
 'app1-username': 'Alice'}

## Utilizing a token

At this point, the server and client have synchronized in agreeing on a token stored as a cookie.  Or rather, really the cookie was merely used as a sideband to the message exchange.  If you like, you can manage cookies more automatically using requests in a manner similar to:

```python
session = requests.Session() 
session.post('https://...', verify='pubkey.pem', data=payload)
answer = some_decoder(session.content)
```

Since we merely store cookies in a local variable, we will need to pass it back explicitly instead.  This is still easy.

In [5]:
import json
resp = requests.get('https://localhost:5001/get-info', 
                    verify='pubkey.pem', cookies=cookies)
print(json.loads(resp.text))

{'Secret-number': 42}


If we do not have correct cookie information that matches the service's database, the action will fail.

In [6]:
cookies['app1-username'] = "Bob"
import json
resp = requests.get('https://localhost:5001/get-info', 
                    verify='pubkey.pem', cookies=cookies)
print(resp.status_code, resp.text)

403 Bob denied access to resource


The verification performed on the server is quite simple.

```python
@app.route('/get-info')
def get_info():
    username = request.cookies.get('app1-username')
    token = request.cookies.get('app1-token')
    if token != auth_tokens.get(username):
        return make_response(f"{username} denied access to resource", 403)
    
    return jsonify({"Secret-number": 42})
```

Obviously, a real-life route will perform some more interesting action, optionally utilizing the cookie data for its specification (and probably POSTing some interesting data to process rather than only using a GET with no query parameters).

Let's let requests manage the cookiejar for us, which will simplify the code even more.

In [7]:
session = requests.Session()
resp = session.post("https://localhost:5001/login", verify='pubkey.pem',
                     data=dict(username="Bob", password="bob_pw"))

for k, v in resp.headers.items():
    print(f"{k}: {v}")
print()
print(resp.status_code, resp.text)

Content-Type: text/html; charset=utf-8
Content-Length: 9
Set-Cookie: app1-username=Bob; Path=/, app1-token=96rCFbBgVl2NL_tbR8C_pg; Path=/, app1-login-timestamp=2021-04-18T23:01:28.825530; Path=/
Server: Werkzeug/1.0.1 Python/3.8.2
Date: Mon, 19 Apr 2021 03:01:28 GMT

200 Logged in


In [8]:
resp = session.get('https://localhost:5001/get-info', verify='pubkey.pem')
print(json.loads(resp.text))

{'Secret-number': 42}


## Next: Authentication protocols

What you have seen already is completely adequate for most web services.  However, sometimes more formal protocols like JSON Web Tokens are used to formalize so-called "claims" between two parties (i.e. client/service or microservice pairs).  If we use that, it also becomes easier to use a private key cryptographic approach rather than a token-based approach.  There are advantages and disadvantages of tokens versus private keys; we look at that in the next lesson.